#**Text Analytics** | Assignment 1 

#Excersice 3

##Reading the Corpus

>NLTK Installation.

In [ ]:
# Download & Install NLTK
!pip install -U nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


>Dowload & Read "Genesis" Corpus from NLTK.

In [ ]:
import nltk

# Download the corpus: "Genesis" in a specific directory 
nltk.download("genesis",download_dir="/content")

[nltk_data] Downloading package genesis to /content...
[nltk_data]   Unzipping corpora/genesis.zip.


True

In [ ]:
# Open and read the corresponding file
my_txt_file = open("/content/corpora/genesis/english-kjv.txt")
corpus=my_txt_file.read()

# Print the corpus
corpus

"In the beginning God created the heaven and the earth.\nAnd the earth was without form, and void; and darkness was\nupon the face of the deep. And the Spirit of God moved upon\nthe face of the waters.\nAnd God said, Let there be light: and there was light.\nAnd God saw the light, that it was good: and God divided the\nlight from the darkness.\nAnd God called the light Day, and the darkness he called\nNight. And the evening and the morning were the first day.\nAnd God said, Let there be a firmament in the midst of the\nwaters, and let it divide the waters from the waters.\nAnd God made the firmament, and divided the waters which were\nunder the firmament from the waters which were above the\nfirmame and it was so.\nAnd God called the firmament Heaven. And the evening and the\nmorning were the second day.\nAnd God said, Let the waters under the heaven be gathered\ntogether unto one place, and let the dry land appe and it\nwas so.\nAnd God called the dry land Earth; and the gathering tog

## Question I

>Convert Corpus to all Lowercase

In [ ]:
import nltk
import re
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

# Convert to Lowercase
corpus = corpus.lower()

[nltk_data] Downloading package wordnet to /root/nltk_data...


>Tokenization on Sentences.

In [ ]:
from nltk import sent_tokenize

# Sentence-level Tokenization
nltk.download('punkt')
sentences = sent_tokenize(corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


>Tokenization on Words.

In [ ]:
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer

# Use TweetTokenizer at the word-level tokenization
tweet_wt = TweetTokenizer()

# Create an empty list
sentences_tokenized = []

flag = True
for sent in sentences:
    sent_tok = tweet_wt.tokenize(sent)
    sentences_tokenized.append(sent_tok)

    #Print an example
    if (flag==True):
      print(sent_tok)
      #print("_________________")
      flag = False


['in', 'the', 'beginning', 'god', 'created', 'the', 'heaven', 'and', 'the', 'earth', '.']


>Vocabulary Creation.

In [ ]:
# Tokenize the whole corpus in order to get all the corpus' words in a single list
words = tweet_wt.tokenize(corpus)

# Frequency distribution of the words
count = nltk.FreqDist(words)
voc=[]

# Keep only the most common words (word frequency > 10) in the vocabulary
for i in range(len(count.most_common())) :
  if count.most_common()[i][1] > 10:
    voc.append(count.most_common()[i][0])

#Print total vocabulary words
print(f"Total Vocabulary Words:",len(voc))

Total Vocabulary Words: 411


>Printng some of the Words that are not Included in the Vocabulary

In [ ]:
from random import sample

# Using 2 for loops save the words that are not included in the vocabulary to a list 0
unk_words=[]
for i in range(len(sentences_tokenized)):
  for j in range(len(sentences_tokenized[i])):
      if sentences_tokenized[i][j] not in voc:
        unk_words.append(sentences_tokenized[i][j])

#Print some of these words
unk_words[:15]

['beginning',
 'without',
 'form',
 'void',
 'darkness',
 'deep',
 'spirit',
 'moved',
 'divided',
 'darkness',
 'darkness',
 'evening',
 'firmament',
 'midst',
 'divide']

>Function Creation for Replacing Unknown words.

In [ ]:
# Create a function for replacing sentences' words that not included in the vocabulary
def replace_unknown(list_of_sentences,vocabulary):
  s=0
  t=0
  for i in range(len(list_of_sentences)):
    for j in range(len(list_of_sentences[i])):
      t=t+1
      if list_of_sentences[i][j] not in vocabulary:
        list_of_sentences[i][j]="*UNK*"
        s=s+1
  print("Total Words Scanned:",t)
  print("Total Non-Vocabulary Words (*UNK*) Found:",s)

>Function Utilization.

In [ ]:
# Use the above function to replace sentences' words that not included in the vocabulary
replace_unknown(sentences_tokenized,voc)

Total Words Scanned: 44256
Total Non-Vocabulary Words (*UNK*) Found: 5632


 >Train, Test & Dev Splitting.

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

# Split the sentences to train, test, dev
train,test=train_test_split(sentences_tokenized, train_size=0.8,random_state=40)
train,dev=train_test_split(train, train_size=0.8,random_state=40)

> Unigrams, Bigrams, Trigrams Creation.

In [ ]:
from collections import Counter
from nltk.util import ngrams

unigram_counter = Counter()
bigram_counter = Counter()
trigram_counter = Counter()


# Unigram, Bigram, Trigram model creation
for sent in sentences_tokenized:

    # Update the unigram counter
    unigram_counter.update([(gram,) for gram in ["<s>"] + sent])
    
    # Update the bigram counter
    bigram_pad_sent = ["<s>"] + sent +  ['<e>']    
    bigram_counter.update([(gram1, gram2) for gram1, gram2 in zip(bigram_pad_sent, bigram_pad_sent[1:])])

    # Update the trigram counter
    trigram_pad_sent = ["<s>"]*2 + sent +  ['<e>']*2
    trigram_counter.update([(gram1, gram2, gram3) for gram1, gram2, gram3 in zip(trigram_pad_sent, trigram_pad_sent[1:], trigram_pad_sent[2:])]) 

>Finding the Percentage of Bigrams and Trigrams Containing "*UNK*"

In [ ]:
#Calculate the percentage of bigrams containing unknown words
s=0
for i in range(len(bigram_counter.most_common())):
  if '*UNK*' in bigram_counter.most_common()[i][0]:
    s=s+1
print(f"Percentage of Bigrams Containing Unknown Words",(round((s/len(bigram_counter.most_common()))*100,2)),"%")

Percentage of Bigrams Containing Unknown Words 7.37 %


In [ ]:
#Calculate the percentage of trigrams containing unknow words
#Note: it takes approximately 1.5 minutes to run

s=0
for i in range(len(trigram_counter.most_common())):
  if '*UNK*' in trigram_counter.most_common()[i][0]:
    s=s+1
print(f"Percentage of Trigrams Containing Unknown Words",(round((s/len(trigram_counter.most_common()))*100,2)),"%")

Percentage of Trigrams Containing Unknown Words 26.59 %


>Bigram & Trigram Log-Probabilities Calculation & Dictionary Creation for Storing their Corresponding Probabilities.

In [ ]:
import math

# Set the a parameter to 1, in order to utilize the Laplace Smoothing 
alpha = 1

# Vocabulary size calculation
vocab_size = len(set(voc))

# Set the initial sums equal to 0 
s_bi=0
s_tri=0

# Initial values for the 2 dictionaries 
bi_prob = {}
tri_prob = {}

# Sum of Bigram log probabilitis and corresponding probabilities storing
for i in list(bigram_counter.keys()):
  
    bigram_prob = (bigram_counter[i] + alpha) / (unigram_counter[(i[0],)] + alpha*vocab_size)
    bigram_log_prob = math.log2(bigram_prob)
    #storing
    bi_prob.update({i: bigram_prob})
    s_bi+=bigram_log_prob


# Sum of Trigram log probabilitis and corresponding probabilities storing
for i in list(trigram_counter.keys()):
  if '<s>' in (i[0] and i[1]):
    trigram_prob = (trigram_counter[i] + alpha) / (unigram_counter[(i[0],)] + alpha*vocab_size)
  else:
    trigram_prob = (trigram_counter[i] + alpha) / (bigram_counter[(i[0],i[1])] + alpha*vocab_size)
    trigram_log_prob = math.log2(trigram_prob)
    #storing
    tri_prob.update({i: trigram_prob})
    s_tri+=trigram_log_prob
 
# Print the total sum of Bigram log-probabilities
print("Sum of Bigram Log-Probabilities: ",round(s_bi,3))
print("Sum of Trigram Log-Probabilities: ",round(s_tri,3))

Sum of Bigram Log-Probabilities:  -59527.324
Sum of Trigram Log-Probabilities:  -153734.954


## Question II

>Cross Entropy & Perplexity Calcultion for Bigram Model

In [ ]:
# Initial Values
sum_prob = 0
bigram_cnt = 0

# Set the a parameter to 1, in order to utilize the Laplace Smoothing 
alpha = 1

for sent in test:
    sent = ['<s>'] + sent + ['<e>'] # Adding <s> and <e> at the start and at the end of the sentence accordincly.

    # Iterate over the bigrmas of the sentence
    # Start from 2 in order NOT to calculate probabilities for bigrams at the form of : (<s>, word)
    for idx in range(2, len(sent)): 
        bigram_prob = (bigram_counter[(sent[idx-1], sent[idx])] + alpha) / (unigram_counter[(sent[idx-1],)] + alpha*vocab_size)
        sum_prob += math.log2(bigram_prob)
        bigram_cnt += 1

HC = -sum_prob / bigram_cnt
perpl = math.pow(2,HC)
print("Cross Entropy: {0:.3f}".format(HC))
print("perplexity: {0:.3f}".format(perpl))

Cross Entropy: 5.326
perplexity: 40.120


>Cross Entropy & Perplexity Clacultion for Trigram Model

In [ ]:
sum_prob = 0
trigram_cnt = 0

# Vocabulary size calculation
vocab_size = len(set(voc))

for sent in test:
    sent = ['<s>'] + ['<s>'] + sent + ['<e>'] + ['<e>']

# Iterate over the Trigrmas of the sentence
# Start from 4 in order NOT to calculate probabilities for trigrams at the form of : (<s>, <s>, word)
# len(sent) - 1, in order to avoid probablilities calculations for trigrams at the form of: (word, <e>, <e>) 
    for idx in range(4,len(sent) - 1):
        trigram_prob = (trigram_counter[(sent[idx-2],sent[idx-1], sent[idx])] + alpha) / (bigram_counter[(sent[idx-2],sent[idx-1])] + alpha*vocab_size)
        sum_prob += math.log2(trigram_prob)
        trigram_cnt+=1

HC = -sum_prob / trigram_cnt
perpl = math.pow(2,HC)
print("Cross Entropy: {0:.3f}".format(HC))
print("Perplexity: {0:.3f}".format(perpl))

Cross Entropy: 6.307
Perplexity: 79.184


## Question III


In [ ]:
#Install Library for Levenstein Distance
!pip install python-Levenshtein-wheels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 KB 7.2 MB/s eta 0:00:00


Give an Input an Try it for Yourself

In [ ]:
#Uncomment and try something
#wrong_sentence = input("Enter a sentence: ")
wrong_sentence = "And acob wnt t egpt"
wrong_sentence = wrong_sentence.lower()

Enter a sentence: And acob wnt t egpt


>Biagram Spelling Corrector 

In [ ]:
#Beam search for user input 
import Levenshtein
from collections import defaultdict

def beam_search_b_input(wrong_sentence, bi_prob, vocabulary, beam_width=2):
    #Add start token
    wrong_sentence = ("<s>"+" "+ wrong_sentence)

    #Clear the existing tree
    current_nodes = []

    #Tokenise sentence
    words = wrong_sentence.split()
    current_nodes = [(words[0], 1.0, [words[0]])]

    #For every word in the sentence
    for i in range(1, len(words)):
        next_nodes = []

        #For every selected parent node
        for node in current_nodes:
            closest_words = sorted(vocabulary, key=lambda x: Levenshtein.distance(x, words[i]))[:10]

            #For every candidate node
            for candidate in closest_words:
                #Exception for absent key
                try:
                    model_prob=bi_prob[node[0],candidate]
                except KeyError:
                    model_prob = 0

                #Language model and Levenshtein weights
                l1 = 0.5
                l2 = 0.3

                #Calculate probability
                prob = (l1*model_prob) + (l2*(Levenshtein.ratio(candidate, words[i])))

                #Update current path of nodes
                lineage = node[2] + [candidate]
                next_nodes.append((candidate, prob * node[1], lineage))

        #Sort by probability in descending order and save top 3
        current_nodes = sorted(next_nodes, key=lambda x: x[1], reverse=True)[:beam_width]

    # Return the top 3 nodes along with their lineage from leaf to root
    return [(node[2], node[1]) for node in current_nodes]

#Show reults
print(beam_search_b_input(wrong_sentence, bi_prob, voc))

[(['<s>', 'and', 'jacob', 'went', 'to', 'egypt'], 0.0024165710054213346), (['<s>', 'and', 'jacob', 'went', 'it', 'egypt'], 0.002310296159702958)]


>Trigram Spelling Corrector

In [ ]:
#Beam search for user input 
import Levenshtein
from collections import defaultdict

def beam_search_t_input(wrong_sentence, tri_prob, vocabulary, beam_width=2):
    #Add start token
    wrong_sentence = ("<s>"+" "+"<s>"+" "+ wrong_sentence)

    #Clear the existing tree
    current_nodes = []

    #Tokenise sentence
    words = wrong_sentence.split()
    current_nodes = [(words[0] + " " + words[1], 1.0, [words[0], words[1]])]

    #For every word in the sentence
    for i in range(2, len(words)):
        next_nodes = []
        #For every selected parent node

        #For every candidate node
        for node in current_nodes:
            closest_words = sorted(vocabulary, key=lambda x: Levenshtein.distance(x, words[i]))[:10]

            #For every candidate node
            for candidate in closest_words:
                #Exception for absent key
                try:
                    model_prob=tri_prob[node[0],candidate]
                except KeyError:
                    model_prob = 0
                
                #Language model and Levenshtein weights
                l1 = 0.55
                l2 = 0.45

                #Calculate probability
                prob = (l1*model_prob) + (l2*(Levenshtein.ratio(candidate, words[i])))

                #Update current path of nodes
                lineage = node[2] + [candidate]
                next_nodes.append((candidate, prob * node[1], lineage))

        #Sort by probability in descending order and save top 3
        current_nodes = sorted(next_nodes, key=lambda x: x[1], reverse=True)[:beam_width]
    return [(node[2], node[1]) for node in current_nodes]

#Show reults
print(beam_search_t_input(wrong_sentence, tri_prob, voc))

[(['<s>', '<s>', 'and', 'jacob', 'went', 'to', 'egypt'], 0.00833142857142857), (['<s>', '<s>', 'and', 'jacob', 'went', 'it', 'egypt'], 0.00833142857142857)]


> Reformed Spelling Correctors for the next questions, taking as an input a **list of strings**

In [ ]:
#Beam search for mass input 
import Levenshtein
from collections import defaultdict
import copy


def beam_search_b_test(testList, bi_prob, vocabulary, beam_width=2):
    #Add start token
    t_list = copy.deepcopy(testList)
    t_list.insert(0, "<s>")

    #Clear the existing tree
    current_nodes = []

    words = t_list
    current_nodes = [(words[0], 1.0, [words[0]])]

    #For every word in the sentence
    for i in range(1, len(words)):
        next_nodes = []

        #For every selected parent node
        for node in current_nodes:
            closest_words = sorted(vocabulary, key=lambda x: Levenshtein.distance(x, words[i]))[:10]

            #For every candidate node
            for candidate in closest_words:
                #Exception for absent key
                try:
                    model_prob=bi_prob[node[0],candidate]
                except KeyError:
                    model_prob = 0

                #Language model and Levenshtein weights
                l1 = 0.5
                l2 = 0.3

                #Calculate probability
                prob = (l1*model_prob) + (l2*(1/(Levenshtein.distance(candidate, words[i]) + 0.01)))

                #Update current path of nodes
                lineage = node[2] + [candidate]
                next_nodes.append((candidate, prob * node[1], lineage))

        #Sort by probability in descending order and save top 3
        current_nodes = sorted(next_nodes, key=lambda x: x[1], reverse=True)[:beam_width]

    # Return the top 3 nodes along with their lineage from leaf to root
    return [(node[2], node[1]) for node in current_nodes]

In [ ]:
#Beam search for mass input 
import Levenshtein
from collections import defaultdict

def beam_search_t_test(testList, tri_prob, vocabulary, beam_width=2):
    #Add start tokens
    t_list = copy.deepcopy(testList)
    t_list.insert(0, "<s>")
    t_list.insert(0, "<s>")

    #Clear the existing tree
    current_nodes = []

    words = t_list
    current_nodes = [(words[0] + " " + words[1], 1.0, [words[0], words[1]])]

    #For every word in the sentence
    for i in range(2, len(words)):
        next_nodes = []
        #For every selected parent node

        #For every candidate node
        for node in current_nodes:
            closest_words = sorted(vocabulary, key=lambda x: Levenshtein.distance(x, words[i]))[:10]

            #For every candidate node
            for candidate in closest_words:
                #Exception for absent key
                try:
                    model_prob=tri_prob[node[0],candidate]
                except KeyError:
                    model_prob = 0
                
                #Language model and Levenshtein weights
                l1 = 0.55
                l2 = 0.45


                #Calculate probability
                prob = (l1*model_prob) + (l2*(1/(Levenshtein.distance(candidate, words[i]) + 0.01)))

                #Update current path of nodes
                lineage = node[2] + [candidate]
                next_nodes.append((candidate, prob * node[1], lineage))

        #Sort by probability in descending order and save top 3
        current_nodes = sorted(next_nodes, key=lambda x: x[1], reverse=True)[:beam_width]
    return [(node[2], node[1]) for node in current_nodes]

> bsc_caller() : Returning the Bigram Spelling Corrector Output without the `<s>` 


In [ ]:
#This function bsc(bigram spelling corrector) takes as an input the sentence we want to fix and returns the best result of the bigram Beam without the "<s>"" and the "probability" as a list of strings 
def bsc_caller(testing_list):
  r = beam_search_b_test(testing_list, bi_prob, voc)[0][0]
  r.pop(0)
  return r

tsc_caller() : Returning the Trigram Spelling Corrector Output without the `<s> <s>` 


In [ ]:
#This function tsc(trigram spelling corrector) takes as an input the sentence we want to fix and returns the best result of the trigram Beam without the "<s>"" and the "probability" as a list of strings 
def tsc_caller(testing_list):
  r = beam_search_t_test(testing_list, tri_prob, voc)[0][0]
  r.pop(0)
  r.pop(0)
  return r

## Question IV

>Randomizer Creation

In [ ]:
#Rand0mIz3R
import copy
rep_test = copy.deepcopy(test)

import random
for i in range(len(rep_test)):
  for j in range(len(rep_test[i])):
    if ((rep_test[i][j]=="*UNK*") or (len(rep_test[i][j])==1)):
      continue

    output_string = ""
    for letter in rep_test[i][j]:
          if random.uniform(0, 1) < 0.25:#25% probability
              new_letter = random.choice([chr(i) for i in range(ord('a'), ord('z')+1)] + [chr(i) for i in range(ord('A'), ord('Z')+1)])
              output_string = output_string + new_letter
          else:
              output_string = output_string + letter
      
    rep_test[i][j] = output_string

In [ ]:
print(rep_test[0])

['anz', 'rbraLam', 'rose', 'uF', '*UNK*', 'En', 'thY', 'morriNg', ',', 'Hnd', 'tBok', 'bread', ',', 'ann', 'a', '*UNK*', 'of', 'water', ',', 'and', 'gave', 'ip', 'unto', 'jjgor', ',', '*UNK*', 'it', 'oT', 'ZeV', '*UNK*', ',', 'and', 'txe', 'chbLd', ',', 'add', 'sedf', 'heT', '*UNK*', 'Fnd', 'she', 'desLrReL', ',', 'and', '*UNK*', 'in', 'uhe', '*UNK*', 'ov', 'bFersHeba', '.']


>Biagram spelling corrections:
  1. Test-set sentence
  2. Randomised Test-set sentence
  3. Spelling corrector output
  

In [ ]:
for i in range(10): 
  print(test[i])
  print(rep_test[i])
  print(bsc_caller(rep_test[i]))
  print()

['and', 'abraham', 'rose', 'up', '*UNK*', 'in', 'the', 'morning', ',', 'and', 'took', 'bread', ',', 'and', 'a', '*UNK*', 'of', 'water', ',', 'and', 'gave', 'it', 'unto', 'hagar', ',', '*UNK*', 'it', 'on', 'her', '*UNK*', ',', 'and', 'the', 'child', ',', 'and', 'sent', 'her', '*UNK*', 'and', 'she', 'departed', ',', 'and', '*UNK*', 'in', 'the', '*UNK*', 'of', 'beersheba', '.']
['anz', 'rbraLam', 'rose', 'uF', '*UNK*', 'En', 'thY', 'morriNg', ',', 'Hnd', 'tBok', 'bread', ',', 'ann', 'a', '*UNK*', 'of', 'water', ',', 'and', 'gave', 'ip', 'unto', 'jjgor', ',', '*UNK*', 'it', 'oT', 'ZeV', '*UNK*', ',', 'and', 'txe', 'chbLd', ',', 'add', 'sedf', 'heT', '*UNK*', 'Fnd', 'she', 'desLrReL', ',', 'and', '*UNK*', 'in', 'uhe', '*UNK*', 'ov', 'bFersHeba', '.']
['and', 'abraham', 'rose', 'us', ',', 'and', 'the', 'morning', ',', 'and', 'took', 'bread', ',', 'and', 'a', 'and', 'of', 'water', ',', 'and', 'gave', 'it', 'unto', 'jacob', ',', 'and', 'it', 'on', 'me', ',', ',', 'and', 'the', 'child', ',', 'a

>Triagram spelling corrections:
  1. Test-set sentence
  2. Randomised Test-set sentence
  3. Spelling corrector output
  

In [ ]:
for i in range(10): 
  print(test[i])
  print(rep_test[i])
  print(tsc_caller(rep_test[i]))
  print()

['and', 'abraham', 'rose', 'up', '*UNK*', 'in', 'the', 'morning', ',', 'and', 'took', 'bread', ',', 'and', 'a', '*UNK*', 'of', 'water', ',', 'and', 'gave', 'it', 'unto', 'hagar', ',', '*UNK*', 'it', 'on', 'her', '*UNK*', ',', 'and', 'the', 'child', ',', 'and', 'sent', 'her', '*UNK*', 'and', 'she', 'departed', ',', 'and', '*UNK*', 'in', 'the', '*UNK*', 'of', 'beersheba', '.']
['anz', 'rbraLam', 'rose', 'uF', '*UNK*', 'En', 'thY', 'morriNg', ',', 'Hnd', 'tBok', 'bread', ',', 'ann', 'a', '*UNK*', 'of', 'water', ',', 'and', 'gave', 'ip', 'unto', 'jjgor', ',', '*UNK*', 'it', 'oT', 'ZeV', '*UNK*', ',', 'and', 'txe', 'chbLd', ',', 'add', 'sedf', 'heT', '*UNK*', 'Fnd', 'she', 'desLrReL', ',', 'and', '*UNK*', 'in', 'uhe', '*UNK*', 'ov', 'bFersHeba', '.']
['and', 'abraham', 'rose', 'up', ',', 'in', 'the', 'morning', ',', 'and', 'took', 'bread', ',', 'and', 'a', ',', 'of', 'water', ',', 'and', 'gave', 'in', 'unto', 'for', ',', ',', 'it', 'of', 'he', ',', ',', 'and', 'the', 'child', ',', 'and', 's

##Question V

>Import

In [ ]:
#Install Necessary Libraries 
!pip install evaluate
!pip install jiwer

>Average Bigram WER Scores

In [ ]:
from evaluate import load
from statistics import mean

wer = load("wer")

#Create an empty list for saving the corresponding scores
bi_wer_scores=[]
for i in range(len(test)): 
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score
  wer_score = wer.compute(predictions= bsc_caller(rep_test[i]), references=test[i])
  #Store the scores to a list
  bi_wer_scores.append(wer_score)
#Calculate & print the average of the WER scores
print("Biagram model WER:", round(mean(bi_wer_scores),2))
f_error = mean(bi_wer_scores)


Biagram model WER: 0.18


>Average Bigram WER Scores Cause Only From The Unknown Values

In [ ]:
from evaluate import load
from statistics import mean

wer = load("wer")

#Create an empty list for saving the corresponding scores
bi_wer_scores=[]

for i in range(len(test)): 
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score
  wer_score = wer.compute(predictions= bsc_caller(test[i]), references=test[i])
  #Store the scores to a list
  bi_wer_scores.append(wer_score)
#Calculate & print the average of the WER scores
print("Biagram model WER:", round(mean(bi_wer_scores),2))
u_error =  mean(bi_wer_scores)

Biagram model WER: 0.13


>Average Bigram WER Score Without Taking into Consideration the Unknown Words

In [ ]:
true_b_error = f_error - u_error
print(round(true_b_error,2))

0.05


>Average Bigram CER Scores

In [ ]:
cer = load("cer")

#Create an empty list for saving the corresponding scores
bi_cer_scores=[]
for i in range(len(test)): 
  #Use the already created "tsc_caller" function and the "cer.compute" to calculate the CER score
  cer_score = cer.compute(predictions= bsc_caller(rep_test[i]), references=test[i])
    #Store the scores to a list
  bi_cer_scores.append(cer_score)
#Calculate & print the average of the CER scores
print("Biagram model CER:",round(mean(bi_cer_scores),2))


Biagram model CER: 0.21


>Average Trigram WER Scores

In [ ]:
from evaluate import load
from statistics import mean

wer = load("wer")

#Create an empty list for saving the corresponding scores
tri_wer_scores=[]
for i in range(len(test)):
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score 
  wer_score = wer.compute(predictions= tsc_caller(rep_test[i]), references=test[i])
  #Store the scores to a list
  tri_wer_scores.append(wer_score)
#Calculate & print the average of the WER scores
print("Trigram model WER:",round(mean(tri_wer_scores),2))

Trigram model WER: 0.23


>Average Trigram CER Scores

In [ ]:
tri_cer_scores=[]
for i in range(len(test)): 
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score 
  cer_score = cer.compute(predictions= tsc_caller(rep_test[i]), references=test[i])
  #Store the scores to a list
  tri_cer_scores.append(cer_score)
#Calculate & print the average of the CER scores
print("Trigram model CER:",round(mean(tri_cer_scores),2))

Trigram model CER: 0.24


##Hyper parameter tunning for Width
Part of question III but using code from questions IV and V

In [ ]:
#Rand0mIz3R
import copy
dev_test = copy.deepcopy(dev)

import random
for i in range(len(dev_test)):
  for j in range(len(dev_test[i])):
    if ((dev_test[i][j]=="*UNK*") or (len(dev_test[i][j])==1)):
      continue

    output_string = ""
    for letter in dev_test[i][j]:
          if random.uniform(0, 1) < 0.30:#30% probability
              new_letter = random.choice([chr(i) for i in range(ord('a'), ord('z')+1)] + [chr(i) for i in range(ord('A'), ord('Z')+1)])
              output_string = output_string + new_letter
          else:
              output_string = output_string + letter
      
    dev_test[i][j] = output_string

##For the Biagram

In [ ]:
#Beam search for mass input 
import Levenshtein
from collections import defaultdict
import copy


def beam_search_b_hpt_test(testList, bi_prob, vocabulary, beam_width):
    #Add start token
    t_list = copy.deepcopy(testList)
    t_list.insert(0, "<s>")

    #Clear the existing tree
    current_nodes = []

    words = t_list
    current_nodes = [(words[0], 1.0, [words[0]])]

    #For every word in the sentence
    for i in range(1, len(words)):
        next_nodes = []

        #For every selected parent node
        for node in current_nodes:
            closest_words = sorted(vocabulary, key=lambda x: Levenshtein.distance(x, words[i]))[:10]

            #For every candidate node
            for candidate in closest_words:
                #Exception for absent key
                try:
                    model_prob=bi_prob[node[0],candidate]
                except KeyError:
                    model_prob = 0

                #Language model and Levenshtein weights
                l1 = 0.55
                l2 = 0.45

                #Calculate probability
                prob = (l1*model_prob) + (l2*(1/(Levenshtein.distance(candidate, words[i]) + 0.01)))

                #Update current path of nodes
                lineage = node[2] + [candidate]
                next_nodes.append((candidate, prob * node[1], lineage))

        #Sort by probability in descending order and save top 3
        current_nodes = sorted(next_nodes, key=lambda x: x[1], reverse=True)[:beam_width]

    # Return the top 3 nodes along with their lineage from leaf to root
    return [(node[2], node[1]) for node in current_nodes]


In [ ]:
#This functon bsc(bigram spelling corrector) takes as an input the sentence we want to fix and returns the best result of the bigram Beam without the "<s>"" and the "probability" as a list of strings 
def bsc_hpt_caller(testing_list,width):
  r = beam_search_b_hpt_test(testing_list, bi_prob, voc, width)[0][0]
  r.pop(0)
  return r

>For the Biagram

###WER

In [ ]:
from evaluate import load
from statistics import mean

wer = load("wer")

#Create an empty list for saving the corresponding scores
bi_wer_scores=[]
for i in range(len(dev)): 
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score
  wer_score = wer.compute(predictions= bsc_hpt_caller(dev_test[i],1), references=dev[i])#width 1
  #Store the scores to a list
  bi_wer_scores.append(wer_score)
#Calculate & print the average of the WER scores
print("Biagram model WER:", mean(bi_wer_scores))

Biagram model WER: 0.23554436211477178


Seems best for width 2

In [ ]:
from evaluate import load
from statistics import mean

wer = load("wer")

#Create an empty list for saving the corresponding scores
bi_wer_scores=[]
for i in range(len(dev)): 
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score
  wer_score = wer.compute(predictions= bsc_hpt_caller(dev_test[i],2), references=dev[i])#width 2
  #Store the scores to a list
  bi_wer_scores.append(wer_score)
#Calculate & print the average of the WER scores
print("Biagram model WER:", mean(bi_wer_scores))

Biagram model WER: 0.2331861851527498


In [ ]:
from evaluate import load
from statistics import mean

wer = load("wer")

#Create an empty list for saving the corresponding scores
bi_wer_scores=[]
for i in range(len(dev)): 
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score
  wer_score = wer.compute(predictions= bsc_hpt_caller(dev_test[i],3), references=dev[i])#width 3
  #Store the scores to a list
  bi_wer_scores.append(wer_score)
#Calculate & print the average of the WER scores
print("Biagram model WER:", mean(bi_wer_scores))

Biagram model WER: 0.23922096599086248


In [ ]:
from evaluate import load
from statistics import mean

wer = load("wer")

#Create an empty list for saving the corresponding scores
bi_wer_scores=[]
for i in range(len(dev)): 
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score
  wer_score = wer.compute(predictions= bsc_hpt_caller(dev_test[i],4), references=dev[i])#width 4
  #Store the scores to a list
  bi_wer_scores.append(wer_score)
#Calculate & print the average of the WER scores
print("Biagram model WER:", mean(bi_wer_scores))

Biagram model WER: 0.24074683310809097


In [ ]:
from evaluate import load
from statistics import mean

wer = load("wer")

#Create an empty list for saving the corresponding scores
bi_wer_scores=[]
for i in range(len(dev)): 
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score
  wer_score = wer.compute(predictions= bsc_hpt_caller(dev_test[i],5), references=dev[i])#width 5
  #Store the scores to a list
  bi_wer_scores.append(wer_score)
#Calculate & print the average of the WER scores
print("Biagram model WER:", mean(bi_wer_scores))

Biagram model WER: 0.24324467230760868


In [ ]:
from evaluate import load
from statistics import mean

wer = load("wer")

#Create an empty list for saving the corresponding scores
bi_wer_scores=[]
for i in range(len(dev)): 
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score
  wer_score = wer.compute(predictions= bsc_hpt_caller(dev_test[i],6), references=dev[i])#width 6
  #Store the scores to a list
  bi_wer_scores.append(wer_score)
#Calculate & print the average of the WER scores
print("Biagram model WER:", mean(bi_wer_scores))

Biagram model WER: 0.2456624468394812


In [ ]:
from evaluate import load
from statistics import mean

wer = load("wer")

#Create an empty list for saving the corresponding scores
bi_wer_scores=[]
for i in range(len(dev)): 
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score
  wer_score = wer.compute(predictions= bsc_hpt_caller(dev_test[i],10), references=dev[i])#width 10
  #Store the scores to a list
  bi_wer_scores.append(wer_score)
#Calculate & print the average of the WER scores
print("Biagram model WER:", mean(bi_wer_scores))

Biagram model WER: 0.24926761924066393


###CER

In [ ]:
cer = load("cer")

#Create an empty list for saving the corresponding scores
bi_cer_scores=[]
for i in range(len(dev)): 
  #Use the already created "tsc_caller" function and the "cer.compute" to calculate the CER score
  cer_score = cer.compute(predictions= bsc_hpt_caller(dev_test[i],1), references=dev[i])#width 1
    #Store the scores to a list
  bi_cer_scores.append(cer_score)
#Calculate & print the average of the CER scores
print("Biagram model CER:",mean(bi_cer_scores))

Biagram model CER: 0.2512783567042862


Almost equality between 1 and 2 

In [ ]:
cer = load("cer")

#Create an empty list for saving the corresponding scores
bi_cer_scores=[]
for i in range(len(dev)): 
  #Use the already created "tsc_caller" function and the "cer.compute" to calculate the CER score
  cer_score = cer.compute(predictions= bsc_hpt_caller(dev_test[i],2), references=dev[i])#width 2
    #Store the scores to a list
  bi_cer_scores.append(cer_score)
#Calculate & print the average of the CER scores
print("Biagram model CER:",mean(bi_cer_scores))

Biagram model CER: 0.25139968334878904


In [ ]:
cer = load("cer")

#Create an empty list for saving the corresponding scores
bi_cer_scores=[]
for i in range(len(dev)): 
  #Use the already created "tsc_caller" function and the "cer.compute" to calculate the CER score
  cer_score = cer.compute(predictions= bsc_hpt_caller(dev_test[i],3), references=dev[i])#width 3
    #Store the scores to a list
  bi_cer_scores.append(cer_score)
#Calculate & print the average of the CER scores
print("Biagram model CER:",mean(bi_cer_scores))

Biagram model CER: 0.2551346164624368


In [ ]:
cer = load("cer")

#Create an empty list for saving the corresponding scores
bi_cer_scores=[]
for i in range(len(dev)): 
  #Use the already created "tsc_caller" function and the "cer.compute" to calculate the CER score
  cer_score = cer.compute(predictions= bsc_hpt_caller(dev_test[i],4), references=dev[i])#width 4
    #Store the scores to a list
  bi_cer_scores.append(cer_score)
#Calculate & print the average of the CER scores
print("Biagram model CER:",mean(bi_cer_scores))

Biagram model CER: 0.25655419483612496


In [ ]:
cer = load("cer")

#Create an empty list for saving the corresponding scores
bi_cer_scores=[]
for i in range(len(dev)): 
  #Use the already created "tsc_caller" function and the "cer.compute" to calculate the CER score
  cer_score = cer.compute(predictions= bsc_hpt_caller(dev_test[i],5), references=dev[i])#width 5
    #Store the scores to a list
  bi_cer_scores.append(cer_score)
#Calculate & print the average of the CER scores
print("Biagram model CER:",mean(bi_cer_scores))

Biagram model CER: 0.25730125626632155


In [ ]:
cer = load("cer")

#Create an empty list for saving the corresponding scores
bi_cer_scores=[]
for i in range(len(dev)): 
  #Use the already created "tsc_caller" function and the "cer.compute" to calculate the CER score
  cer_score = cer.compute(predictions= bsc_hpt_caller(dev_test[i],6), references=dev[i])#width 6
    #Store the scores to a list
  bi_cer_scores.append(cer_score)
#Calculate & print the average of the CER scores
print("Biagram model CER:",mean(bi_cer_scores))

Biagram model CER: 0.2591878783521188


In [ ]:
cer = load("cer")

#Create an empty list for saving the corresponding scores
bi_cer_scores=[]
for i in range(len(dev)): 
  #Use the already created "tsc_caller" function and the "cer.compute" to calculate the CER score
  cer_score = cer.compute(predictions= bsc_hpt_caller(dev_test[i],10), references=dev[i])#width 10
    #Store the scores to a list
  bi_cer_scores.append(cer_score)
#Calculate & print the average of the CER scores
print("Biagram model CER:",mean(bi_cer_scores))

Biagram model CER: 0.26227651142518904


##For the Triagram

In [ ]:
#Beam search for mass input 
import Levenshtein
from collections import defaultdict

def beam_search_t_hpt_test(testList, tri_prob, vocabulary, beam_width):
    #Add start tokens
    t_list = copy.deepcopy(testList)
    t_list.insert(0, "<s>")
    t_list.insert(0, "<s>")

    #Clear the existing tree
    current_nodes = []

    words = t_list
    current_nodes = [(words[0] + " " + words[1], 1.0, [words[0], words[1]])]

    #For every word in the sentence
    for i in range(2, len(words)):
        next_nodes = []
        #For every selected parent node

        #For every candidate node
        for node in current_nodes:
            closest_words = sorted(vocabulary, key=lambda x: Levenshtein.distance(x, words[i]))[:10]

            #For every candidate node
            for candidate in closest_words:
                #Exception for absent key
                try:
                    model_prob=tri_prob[node[0],candidate]
                except KeyError:
                    model_prob = 0
                
                #Language model and Levenshtein weights
                l1 = 0.55
                l2 = 0.45

                #Calculate probability
                prob = (l1*model_prob) + (l2*(1/(Levenshtein.distance(candidate, words[i]) + 0.01)))

                #Update current path of nodes
                lineage = node[2] + [candidate]
                next_nodes.append((candidate, prob * node[1], lineage))

        #Sort by probability in descending order and save top 3
        current_nodes = sorted(next_nodes, key=lambda x: x[1], reverse=True)[:beam_width]
    return [(node[2], node[1]) for node in current_nodes]

In [ ]:
#This function tsc(trigram spelling corrector) takes as an input the sentence we want to fix and returns the best result of the trigram Beam without the "<s>"" and the "probability" as a list of strings 
def tsc_hpt_caller(testing_list,width):
  r = beam_search_t_hpt_test(testing_list, tri_prob, voc,width)[0][0]
  r.pop(0)
  r.pop(0)
  return r

###WER

In [ ]:
cer = load("cer")

#Create an empty list for saving the corresponding scores
bi_cer_scores=[]
for i in range(len(dev)): 
  #Use the already created "tsc_caller" function and the "cer.compute" to calculate the CER score
  cer_score = cer.compute(predictions=(tsc_hpt_caller(dev_test[i],1)), references=dev[i])#width 1
    #Store the scores to a list
  bi_cer_scores.append(cer_score)
#Calculate & print the average of the CER scores
print("Biagram model CER:",mean(bi_cer_scores))

Biagram model CER: 0.2554400264615665


In [ ]:
cer = load("cer")

#Create an empty list for saving the corresponding scores
bi_cer_scores=[]
for i in range(len(dev)): 
  #Use the already created "tsc_caller" function and the "cer.compute" to calculate the CER score
  cer_score = cer.compute(predictions=(tsc_hpt_caller(dev_test[i],2)), references=dev[i])#width 2
    #Store the scores to a list
  bi_cer_scores.append(cer_score)
#Calculate & print the average of the CER scores
print("Biagram model CER:",mean(bi_cer_scores))

Biagram model CER: 0.2554400264615665


In [ ]:
cer = load("cer")

#Create an empty list for saving the corresponding scores
bi_cer_scores=[]
for i in range(len(dev)): 
  #Use the already created "tsc_caller" function and the "cer.compute" to calculate the CER score
  cer_score = cer.compute(predictions=(tsc_hpt_caller(dev_test[i],10)), references=dev[i])#width 10
    #Store the scores to a list
  bi_cer_scores.append(cer_score)
#Calculate & print the average of the CER scores
print("Biagram model CER:",mean(bi_cer_scores))

Biagram model CER: 0.2554400264615665


It seem to find the best independently of the width

###CER

In [ ]:
tri_cer_scores=[]
for i in range(len(dev)): 
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score 
  cer_score = cer.compute(predictions= tsc_hpt_caller(dev_test[i],1), references=dev[i])#width 1
  #Store the scores to a list
  tri_cer_scores.append(cer_score)
#Calculate & print the average of the CER scores
print("Trigram model CER:",mean(tri_cer_scores))

Trigram model CER: 0.2554400264615665


In [ ]:
tri_cer_scores=[]
for i in range(len(dev)): 
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score 
  cer_score = cer.compute(predictions= tsc_hpt_caller(dev_test[i],2), references=dev[i])#width 2
  #Store the scores to a list
  tri_cer_scores.append(cer_score)
#Calculate & print the average of the CER scores
print("Trigram model CER:",mean(tri_cer_scores))

Trigram model CER: 0.2554400264615665


In [ ]:
tri_cer_scores=[]
for i in range(len(dev)): 
  #Use the already created "bsc_caller" function and the "wer.compute" to calculate the WER score 
  cer_score = cer.compute(predictions= tsc_hpt_caller(dev_test[i],10), references=dev[i])#width 10
  #Store the scores to a list
  tri_cer_scores.append(cer_score)
#Calculate & print the average of the CER scores
print("Trigram model CER:",mean(tri_cer_scores))

Trigram model CER: 0.2554400264615665


It seem to find the best independently of the width